# Analysis of Automobile Inventory Data

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import calendar
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report

### CURR and PARENT directory constants

In [2]:
CURR_DIR = os.path.dirname(os.path.abspath('__file__'))
PARENT_DIR = os.path.abspath(os.path.join(CURR_DIR, os.pardir))

### Import dataset ignoring headers

In [3]:
df = pd.read_excel(PARENT_DIR + '\\assets\\data\\items_analysis.xlsx', sheet_name='Sheet1')

### Dataset

In [4]:
df.head()

,Item_Number,Make_Or_Buy_Flag,Minimum_Order_Quantity,Standard_Cost,Ordered_Quantity,Month,Request_Date,Cumulative LT
0,066.01.374.1.37,Make,NaN,1027.19182,2,2015-08,2015-08-24 00:00:00,76.986458
1,066.322.002,Buy,5.0,893.80000,2,2015-01,2015-01-23 00:00:00,156.000000
2,066.322.002,Buy,5.0,893.80000,2,2015-01,2014-12-22 15:09:00,156.000000
3,066.322.002,Buy,5.0,893.80000,4,2015-09,2015-09-16 00:00:00,156.000000
4,066.322.002,Buy,5.0,893.80000,2,2015-02,2015-01-28 00:00:00,156.000000


In [5]:
# Convert uppercase to lowercase and replace empty spaces with underscore
df.columns = [i.replace(" ", "_").lower() for i in df.columns]
df.columns

Index(['item_number', 'make_or_buy_flag', 'minimum_order_quantity',
       'standard_cost', 'ordered_quantity', 'month', 'request_date',
       'cumulative_lt'],
      dtype='object')

In [6]:
# Convert 'month' and 'reqeust date' to datetime and remove time by normalization
df['month'] = pd.to_datetime(df['month'])
df['month'] = df['month'].dt.month.apply(lambda x: calendar.month_abbr[x])
df['request_date'] = (pd.to_datetime(df['request_date'])).dt.normalize()
# df['date'] = df['request_date'].dt.day
df['year'] = df['request_date'].dt.year

In [7]:
# Create 'Buy' and 'Make' dataframe and remove 'make_or_buy_flag'
df_buy = df[df['make_or_buy_flag'] == 'Buy'].drop(['make_or_buy_flag', 'minimum_order_quantity', 'standard_cost', 'cumulative_lt'], axis=1)
df_buy = df_buy.fillna(df_buy.mean().astype(int))

df_make = df[df['make_or_buy_flag'] == 'Make'].drop(['make_or_buy_flag', 'minimum_order_quantity'], axis=1)
df_make = df_make.fillna(df_make.mean().astype(int))

In [8]:
# Grouping by Item, Year, Month
df_buy_group = round(df_buy.groupby(['item_number', 'year', 'month']).mean().astype(int))

In [14]:
item = df_buy_group.index.levels[0]
print(df_buy_group.index.get_level_values('item_number').unique())
# print(df_buy_group.index.get_level_values('year'))

Index(['066.322.002', '066.322.006', '066.385.001', '066.563.002',
       '066.565.001', '066.565.005', '066.570.002', '066.570.003',
       '066.577.001', '066.577.003',
       ...
       '942.900.117', '942.900.118', '942.900.120', '942.900.121',
       '942.900.122', '942.900.127', '942.900.160', '942.900.166',
       '942.900.176', '942.900.177'],
      dtype='object', name='item_number', length=102)


In [13]:
# Importing the dataset
X = df_buy.iloc[:, 2].values
y = df_buy.iloc[:, 3].values

In [11]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=0)
X_train, X_test, y_train, y_test = X_train.reshape(-1, 1), X_test.reshape(-1, 1), y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [12]:
# Fitting Simple Linear Regression to the Training set
regressor = LinearRegression()
regressor.fit(X_train, y_train)

ValueError: could not convert string to float: 'Mar'

In [ ]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [ ]:
# Visualising the Training set results
plt.scatter(X_train, y_train, color='red')
plt.plot(X_train, regressor.predict(X_train), color='blue')
plt.title('Standard Cost vs Order Quantity (Training set)')
plt.xlabel('Order Quantity')
plt.ylabel('Standard Cost')
plt.show()

In [ ]:
# Visualising the Training set results
plt.scatter(X_test, y_test, color='red')
plt.plot(X_train, regressor.predict(X_train), color='blue')
plt.title('Standard Cost vs Order Quantity (Test set)')
plt.xlabel('Order Quantity')
plt.ylabel('Standard Cost')
plt.show()